In [99]:
import world
import agents
from ff import Solved
import ff
import pddl_functions
from teacher import TeacherAgent
import os
from collections import namedtuple, defaultdict, Counter
import random
import goal_updates

In [2]:
w = world.PDDLWorld('blocks-domain.pddl', 'onerule/problem26.pddl')
w2 = world.PDDLWorld('blocks-domain.pddl', 'onerule/problem27.pddl')
teacher = TeacherAgent()
agent = agents.CorrectingAgent(w, teacher=teacher)



Parsing domain blocks-domain.pddl
Parsing problem onerule/problem26.pddl
Parsing domain blocks-domain.pddl
Parsing problem onerule/problem27.pddl


In [139]:
RuleBelief = namedtuple('RuleBelief', ['colours', 'rule1', 'rule2', 'belief'])

def create_belief_dict(prior=0.001):
    # [[(r1,r2), (r1,-r2)],[(-r1,r2), (-r1, -r2)]]
    return np.array([[prior*prior, prior*(1-prior)], [(1-prior)*prior, (1-prior)**2]])

class RuleBelief(object):

    def __init__(self, colours, rule1, rule2, prior=0.001):
        # [[(r1,r2), (r1,-r2)],[(-r1,r2), (-r1, -r2)]]
        self.colours = colours
        self.rule1 = rule1
        self.rule2 = rule2
        self.belief = np.array([[prior*prior, prior*(1-prior)], [(1-prior)*prior, (1-prior)**2]])
    
    def p_r1(self):
        return np.sum(self.belief, axis=1)[0]
    
    def p_r2(self):
        return np.sum(self.belief, axis=0)[0]
    
    def update(self, message_probs):
        # message_probs: [P(m=r1|x), P(m=r2|x)]
        m_r1, m_r2 = message_probs
        self.belief = np.array(
        [[1*self.p_r2()*m_r1 + 1*self.p_r1()*m_r2,     1*(1-self.p_r2())*m_r1 + 0*self.p_r1()*m_r2],
         [0*self.p_r2()*m_r1 + 1*(1-self.p_r1())*m_r2, 0*(1-self.p_r2())*m_r1 + 0*(1-self.p_r1())*m_r2]])
        
        
def generate_rulebelief(colours, rule1, rule2, prior=0.001):
    return RuleBelief(colours, rule1, rule2, create_belief_dict(prior=prior))

rule1, rule2 = goal_updates.create_goal_options(['red'], ['blue'])
rb = RuleBelief(('red', 'blue'), rule1, rule2)



In [156]:
rule1, rule2 = goal_updates.create_goal_options(['red'], ['blue'])
rb = RuleBelief(('red', 'blue'), rule1, rule2)
print(np.sum(rb.belief))
print(rb.p_r1())
print(rb.p_r2())
print(rb.belief[0,0])
rb.update((0.7, 0.3))
print(np.sum(rb.belief))
print(rb.p_r1())
print(rb.p_r2())
print(rb.belief[0,0])
rb.update((0, 1))
print(np.sum(rb.belief))
print(rb.p_r1())
print(rb.p_r2())
print(rb.belief[0,0])

1.0
0.001
0.001
1e-06
0.9999999999999999
0.7002999999999999
0.30069999999999997
0.001
1.0
0.7002999999999999
1.0
0.7002999999999999


In [127]:
np.sum([[1,2],[3,4]], axis=1)

array([3, 7])

In [111]:
True True : goal_belief()


    
def get_prior(index, belief):
    
    
def update_goal_belief(message_beliefs, goal_belief):
    m1, m2 = message_beliefs
    for r1 in [True, False]:
        for r2 in [True, False]:
            {(r1, r2) = }

update_goal_belief((0.7, 0.3), rb.belief)

SyntaxError: invalid syntax (<ipython-input-111-67266181ae00>, line 1)

In [9]:
d12 = {i:1/12 for i in range(1, 13)}
_2d6 = defaultdict(float)
for i in range(6):
    i = i+1
    for j in range(6):
        j = j+1
        _2d6[i+j] += 1
_2d6 = {k:v/36 for k,v in _2d6.items()}


In [12]:
random.random()

0.9727324207989289

In [17]:
import numpy as np

In [35]:
class RV(object):
    def __init__(self, outcomes, p):
        self.outcomes = np.array(outcomes)
        self.p = np.array(p)
    
    def sample(self):
        u = np.random.rand()
        cumulative = 0
        for i, p in enumerate(self.p):
            cumulative += p
            if u < cumulative:
                return self.outcomes[i]
    
    def expectation(self):
        expect = np.sum([value*p for value, p in zip(self.outcomes, self.p)])
        return expect

In [23]:
d12_outcomes = np.array(range(1,13))
d12_ps = np.array([1/12]*12)

In [57]:
d6x2_outcomes = np.array(range(2, 13))

d6x2_ps = np.array(list(Counter([i + j for i in range(1, 7) for j in range(1, 7)]).items()))[:,1]/36

In [58]:
d12 = RV(d12_outcomes, d12_ps)
d6x2 = RV(d6x2_outcomes, d6x2_ps)

In [61]:
print(d12.expectation())
print(d6x2.expectation())

6.5
6.999999999999998


In [28]:
for i in np.array([1/12]*12):
    print(i)

0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333
0.08333333333333333


In [60]:
d6x2.expectation()

6.999999999999998

In [68]:
reroll_d12_ps = [2/12*1/12]+ [1/12 + 2/12*1/12]*10

In [65]:
reroll_d12 = RV(d12_outcomes, reroll_d12_ps)
print(reroll_d12.expectation())

6.333333333333332


In [67]:
np.sum(reroll_d12.p)

0.986111111111111

In [75]:
data = []
for i in range(10000):
    s = d12.sample()
    if s < 3:
        s = d12.sample()
    data.append(s)

np.mean(data)

7.3098

In [71]:
d6_outcomes = range(1, 7)
d6_ps = [1/6]*6
d6 = RV(d6_outcomes, d6_ps)

In [76]:
data = []
for i in range(10000):
    s1 = d6.sample()
    s2 = d6.sample()
    if s1 < 3:
        s1 = d6.sample()
    if s2 < 3:
        s2 = d6.sample()
    data.append(s1+s2)
np.mean(data)

8.3621

In [82]:
a= [2]
a.remove(1)
a

ValueError: list.remove(x): x not in list

In [79]:
fugitive game 

SyntaxError: invalid syntax (<ipython-input-79-dbd72e29d9b4>, line 1)

In [98]:
class Hand(object):
    def __init__(self, cards):
        self.cards = cards
    
    def add_card(self, card):
        self.cards.append(card)
    
    def remove_card(self, card):
        self.cards.remove(card)
        
class Deck(object):
    def __init__(self, cards):
        self.cards = cards
        random.shuffle(self.cards)
    def draw(self):
        return self.cards.pop()
    
class GameState(object):
    
    def __init__(self):
        self.deck1 = Deck(list(range(4, 14+1)))
        self.deck2 = Deck(list(range(15, 28+1)))
        self.deck3 = Deck(list(range(29, 41+1)))
        self.decks = [self.deck1, self.deck2, self.deck3]
        self.hideouts = [0]
        self.revealed_hideouts = [0]
        self.fugitive_hand = Hand([1, 2, 3, 42])
        self.detective_hand = Hand([])
        


class Fugitive(object):
    
    def __init__(self, game):
        self.hand = Hand([1, 2, 3, 42])
        self.game = game

    def show_hand(self):
        print(' '.join(self.hand))
        
    def draw_card(self, deck):
        self.hand.add_card(self.game.decks[deck].draw())
    
    def play_hideout(self, card):
        self.hand.remove_card(card)
        
    
        
        
class Detective(object):
    
    def __init(self, game):
        self.hand = Hand([])
        self.game = game

In [96]:
d = Deck(list(range(1, 14)))

In [97]:
d.draw()

12

In [159]:
np.argmax([[1, 2], [3, 4]])

3

In [160]:
np.

TypeError: cannot create 'numpy.flatiter' instances

In [161]:
a = np.array([[1, 2], [3, 4]])

In [165]:
a = np.array([[1, 2], [7, 4]])
a.flatten()[np.argmax(a)]

7

In [168]:
a = []
a.extend(np.array([1,2,3]))
a

[1, 2, 3]